# Практика
Используемые библиотеки

In [2]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [3]:
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = get_content(url)
df=pd.read_csv(io.StringIO(s.decode('UTF8')))


### Если не работает загрузка on-line
#df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [7]:
import re
# my_path = r'C:\Users\11137731\Documents\DS\Python\HW-21'
# df_h = pd.read_csv(f'{my_path}\\Химический анализ родника в Нескучном саду.csv',sep = ';')
df_h = pd.read_csv('Химический анализ родника в Нескучном саду.csv',sep = ';')
df_h.set_index('Показатель')

def assess(strin, parm):
    min_p = 0
    max_p = 0
    itog = 'нейтрально'

    if len(re.findall('[а-я]',parm.lower()))/len(parm) > 0.7:
        parin = 0.0
    else: 
        parin = float(parm.replace(' ','').replace(',','.'))

    if strin.find('не более') != -1 :
        srt = strin.replace(' ','').replace(',','.')[7:]
        if srt.find('-') != -1:
            srt =  srt.split('-')[1]
        max_p = float(srt)
        if parin <= max_p :
            itog = 'норма'
        else :
            itog = 'выше нормы'

    elif strin.find('не менее') != -1:
        srt = float(strin.replace(' ','').replace(',','.')[7:])
        if parin >= min_p :
            itog = 'норма'
        else :
            itog = 'ниже нормы'
        
    elif strin.find('в пределах') != -1:
        srt = strin.replace(' ','').replace(',','.')[9:]
        min_p = float(srt.split('-')[0])
        max_p = float(srt.split('-')[1])
        if parin >= min_p and parin <= max_p :
            itog = 'норма'
        else:
            if parin <= min_p :
                itog = 'ниже нормы'
            else:
                itog = 'выше нормы'
    return itog

df_h['Оценка'] = df_h.apply(lambda x: assess(x['Норматив'], x['Результат анализа']),axis=1)
display(df_h)

,Показатель,Единица измерений,Результат анализа,Норматив,Оценка
0,pH,единицы pH,8.4,в пределах 6-9,норма
1,Запах,баллы,1,не более 2-3,норма
2,Цветность,градусы,б/цвета,не более 30,норма
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,норма
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",норма
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",норма
6,Нитраты (по NO3),мг/дм3,24,не более 45,норма
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",норма
8,Хлориды (Cl),мг/дм3,200,не более 350,норма
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,норма


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [10]:
# Ваше решение
import pandas as pd
import random
B = ['orange'] * 5 + ['apple'] * 4

n = 100000
A = pd.DataFrame(list(map(lambda a: random.choices(B, k = 3), range(n))), columns = ['f1','f2','f3'])
print(len(A.query('f1 == "orange" & f2 == "orange" & f3 == "orange"'))/n)

0.17122


Ответ 0,119 эксперементально не подтвердился.

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [11]:
# Ваше решение
res = []
n = 100000
for i in range(n):
    count = 1
    B2 = ['standart'] * 7 +['not-standart'] * 3
    while True :
        det = random.choice(B2)
        if det == 'standart':
            res.append(count)
            break
        else:
            count += 1
            B2.remove(det)
res.count(2)/n

0.2366

Ответ 7/30 или 0.23333 эксперементально подтвердился.